In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from gensim.parsing.preprocessing import remove_stopwords
from nltk import sent_tokenize
from keybert import KeyBERT
import pandas as pd
import numpy as np
import spacy
import time
import copy
import re

In [2]:
nlp = spacy.load("en_core_web_sm")
data = pd.read_csv('Index_words_eng.csv')
data.head()

,Index,Word
0,0,accessiblecomputing
1,1,anarchism
2,2,afghanistanhistory
3,3,afghanistangeography
4,4,afghanistanpeople


In [3]:
INDEX = [0, 50]
n = list(data.Index)[INDEX[0] : INDEX[1]]

In [4]:
def append_corpus(index: list or tuple =[0,10], corpus:list = []) -> None:
    '''Чтение файлов с текстами собирая их в масси для последущих манипуляций'''
    index = list(data.Index)[index[0]: index[1]]
    for i in index:
        with open(f"dataset_eng/{i}.txt", 'r') as fl:
            text = fl.read().lower()
            text = re.sub(r'/(style=")([a-zA-Z0-9:;\.\s\(\)\-\,]*)(")/gi', '', text)
            text = re.sub(r'ref|url|link|title|aa|url', '', text)
            text = re.sub(r"title", '', text)
            text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»""‘’]))', '', text)
            text = re.sub(r"[^a-zA-Z\.]+", " ", text)
       
        corpus.append(remove_stopwords(text))

In [5]:
def key_words(corpus) -> list:
    """Выделение ключевых слов с использованием BERT"""
    res = []
    for text in corpus:
        try:
            model = KeyBERT('distilbert-base-nli-mean-tokens')
            keywords = model.extract_keywords(text, keyphrase_length=2, use_maxsum=True)
            [keywords.append(i) for i in model.extract_keywords(text, keyphrase_length=1, use_maxsum=True)];
        except Exception as e:
            #print(e)
            info = e
            keywords = ['None' ] * 10
        res.append(keywords)
    return res

In [6]:
def create_DataFrame(data:pd.DataFrame, result, index) -> pd.DataFrame:
    return pd.DataFrame({i:j for i,j in zip(list(data.loc[index[0]:index[1], 'Word']), result)})

In [7]:
def n_gram_tfidf(corpus, res, n= 4) -> None:
    """ИСпользование n-Grams & TF-IDf"""
    for index_text, text in enumerate(corpus):
        size = (1,1)
        text = re.sub(r"shell|redirect|category", ' ', text)
        data = []
        
        
        x2_variaons = {i : TfidfVectorizer(ngram_range = (i, i)).fit_transform(corpus)  for i in range(2, n+1)}
        
        
        for i in range(2, n + 1):
            size = (i, i)
            vectorizer = CountVectorizer(ngram_range =size) 
            try:
                X1 = vectorizer.fit_transform(sent_tokenize(text))  
                features = (vectorizer.get_feature_names()) 

                # Applying TFIDF 
                # You can still get n-grams here 
#                 vectorizer = TfidfVectorizer(ngram_range = size) 
#                 X2 = vectorizer.fit_transform(corpus)
                X2 = x2_variaons[i]
#                 scores = (X2.toarray()) 

                # Getting top ranking features 
                sums = X2.sum(axis = 0) 

                for col, term in enumerate(features): 
                    data.append( (term, sums[0, col] )) 
            except Exception as e:
                info = e
                #print(e)
        ranking = pd.DataFrame(data, columns = ['term', 'rank']) 
        words = (ranking.sort_values('rank', ascending = False)) 
        [res[index_text].append(_) for _ in list(words.head().term.values)]

In [8]:
def alignment_of_pillars(example_output) -> None:
    """Выравнивание всех колонок по длине для построение датафрейма"""
    for index, i in enumerate(example_output):
        if len(i) < max([len(i) for i in example_output]):
            while len(i)< max([len(i) for i in example_output]):
                example_output[index].append('None')

In [9]:
def group_crossing(arg:list) -> list:
    """Группировка ключевых n-Gramm тайтла по общему слову"""
    tmp = []
    for i in arg:
        test = re.sub(r'shell|category|redirect', '', i)
        if len(test) > 0:
            if len(test.split() ) == 1:
                tmp.append(re.sub(r'\s+', '', test))
            else :
                tmp.append(test)
    tmp = list(set(tmp))
    result_new = []
    for index, el in enumerate(tmp):
        test = []
        for index_, el_ in enumerate(tmp):
            if el_ != 'None' and len(set(el.split()) & set(el_.split())) > 0 :
                test.append(el_)
                tmp[index_] = 'None'
        if len(test) > 0 :
            if len(test) > 1 :
                result_new.append(set(test))
            else:
                result_new.append(test[0])
    return result_new

In [10]:
def recognazie_spam(array_words:list) -> list:
    """Поиск неевалидной информации(которая не относится к терминам)"""
    result = []
    for  el in array_words:
        flag = True
        if type(el) == set:
            text = " ".join(el)
            doc = nlp(text)
            for ent in doc.ents:
                flag = False
                break
        if flag:
            result.append(el)
    return result

In [11]:
from_ = 0
for to in np.arange(50, len(n)+ 1, 50):
    start_time = time.time()
    
    corpus = []
    index= [from_, to]
    
    append_corpus(index=index, corpus=corpus)
    
    print("--- %s append_corpus ---" % (time.time() - start_time))
    start_time = time.time()

    
    corpus_with_key_words = key_words(corpus)
    
    print("--- %s corpus_with_key_words ---" % (time.time() - start_time))
    start_time = time.time()
    
    
    n_gram_tfidf(corpus=corpus, res = corpus_with_key_words)
    
    print("--- %s n_gram_tfidf ---" % (time.time() - start_time))
    start_time = time.time()
    
    for index, el  in enumerate(corpus_with_key_words):
        corpus_with_key_words[index] = recognazie_spam(group_crossing(el))
    
    print("--- %s recognazie_spam ---" % (time.time() - start_time))

    
    print(from_, to)
    from_= to

--- 0.06178903579711914 append_corpus ---
--- 78.23599338531494 corpus_with_key_words ---
--- 7.465719699859619 n_gram_tfidf ---
--- 0.4700205326080322 recognazie_spam ---
0 50


In [12]:
example_output = copy.deepcopy(corpus_with_key_words)

In [13]:
alignment_of_pillars(example_output=example_output)

In [14]:
data = create_DataFrame(data=data, result=example_output, index=[INDEX[0], INDEX[1]])

In [15]:
data.head()

,accessiblecomputing,anarchism,afghanistanhistory,afghanistangeography,afghanistanpeople,afghanistancommunications,afghanistantransportations,afghanistanmilitary,afghanistantransnationalissues,assistivetechnology,...,aynrand,alexanderthegreat,anchoragealaska,argumentforms,argumentsfortheexistenceofgod,anarchy,asciiart,academyawards,academyawards/bestpicture,austrialanguage
0,"{camelcase unprintworthy, accessibility camelc...",long roderick,history,None,demographics,communications,None,camelcase,"{relations, foreign relations}","{technology, technology camelcase, assistive t...",...,ayn,None,None,"{logical form camelcase, logical form, form, l...","{existence god camelcase, existence god, god c...","{anarchy, anarchy camelcase}",ascii,None,"{subpage, best picture camelcase subpage, came...",None
1,None,michael editor stears,camelcase,None,camelcase,camelcase,None,None,camelcase,camelcase,...,None,None,None,None,None,camelcase,camelcase,None,"{best picture, best}",None
2,None,capitalist,None,None,None,None,None,None,None,assistive,...,None,None,None,None,None,None,None,None,camelcase,None
3,None,liberalism,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,century anarchists,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [16]:
corpus_with_key_words[1]

['long roderick',
 'michael editor stears',
 'capitalist',
 'liberalism',
 'century anarchists',
 'postcolonialism',
 'counterculture',
 'fascists']